In [2]:
import sys
import time
import os
import tensorflow as tf
import numpy as np

from collections import namedtuple
from tensorflow.python import debug as tf_debug
from tensorflow.core.example import example_pb2
from importlib import reload

import json, os, re, shutil, sys, time
import collections, itertools
from rouge import Rouge


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import glob
import random
import struct
import csv
import batch; reload(batch)
import data; reload(data)
import model; reload(model)

from batch import Example,Batch
from batcher import Batcher
from data import Vocab
from model import SummarizationModel
from decode import BeamSearchDecoder
import training_util as tutil



In [5]:
def setup_decode_params():
    hps_dict = {
        'mode' : 'decode',
        'single_pass' : True,
        'log_root' : '/home/ubuntu/W266/final_0/W266_Final/model_3/saved',
        'cp_dir':'train',
        'hidden_dim' : 200,
        'emb_dim' : 128,
        'batch_size' : 4,     #<---------- Keep same as beam size
        'max_enc_steps' : 400,
        'max_dec_steps' : 1,        #<---------------Keep at 1
        'beam_dec_steps' : 120, # Decode steps during beam search
        'beam_size' : 4,
        'min_dec_steps' : 35,
        'vocab_size' : 50000,
        'lr' : 0.15,
        'adagrad_init_acc' : 0.1,
        'rand_unif_init_mag' : 0.02,
        'trunc_norm_init_std' : 1e-4,
        'max_grad_norm' : 2.0,
        'pointer_gen' : False,
        'coverage' : False,
        'cov_loss_wt' : 1.0,
        'vocab_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_processed/vocab',
        'data_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_chunked/validation_*'
    }
    
    hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)
    return hps

In [6]:
hps = setup_decode_params()
vocab = Vocab(hps.vocab_path, hps.vocab_size)
batches = tutil.get_decode_data(hps,vocab,hps.data_path,False)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 31193
INFO:tensorflow:[TOTAL Examples] : 31193
INFO:tensorflow:Creating batches..COMPLETE


In [ ]:
tf.reset_default_graph()
model_params = dict(hps=hps, 
                    vocab=vocab)

lm = SummarizationModel(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()
lm.summarizeGraph()

with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    decoder = BeamSearchDecoder(lm, session, vocab, hps, saver)
    decoder.decode(batches) 

INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 1
INFO:tensorflow:Building core graph...COMPLETE
INFO:tensorflow:Building Sampler graph...
INFO:tensorflow:Building Sampler graph...COMPLETE
INFO:tensorflow:Loading checkpoint /home/ubuntu/W266/final_0/W266_Final/model_3/saved/train/model-37122
INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_3/saved/train/model-37122
INFO:tensorflow:Added 0 to file
INFO:tensorflow:Added 1 to file
INFO:tensorflow:Added 2 to file
INFO:tensorflow:Added 3 to file
INFO:tensorflow:Added 4 to file
INFO:tensorflow:Added 5 to file
INFO:tensorflow:Added 6 to file
INFO:tensorflow:Added 7 to file
INFO:tensorflow:Added 8 to file
INFO:tensorflow:Added 9 to file
INFO:tensorflow:Added 10 to file
INFO:tensorflow:Added 11 to file
INFO:tensorflow:Added 12 to file
INFO:tensorflow:Added 13 to file
INFO:tensorflow:Added 14 to file
INFO:tensorflow:Added 15 to file
INFO:tensorflow:Added 16 to file

INFO:tensorflow:Added 231 to file
INFO:tensorflow:Added 232 to file
INFO:tensorflow:Added 233 to file
INFO:tensorflow:Added 234 to file
INFO:tensorflow:Added 235 to file
INFO:tensorflow:Added 236 to file
INFO:tensorflow:Added 237 to file
INFO:tensorflow:Added 238 to file
INFO:tensorflow:Added 239 to file
INFO:tensorflow:Added 240 to file
INFO:tensorflow:Added 241 to file
INFO:tensorflow:Added 242 to file
INFO:tensorflow:Added 243 to file
INFO:tensorflow:Added 244 to file
INFO:tensorflow:Added 245 to file
INFO:tensorflow:Added 246 to file
INFO:tensorflow:Added 247 to file
INFO:tensorflow:Added 248 to file
INFO:tensorflow:Added 249 to file
INFO:tensorflow:Added 250 to file
INFO:tensorflow:Added 251 to file
INFO:tensorflow:Added 252 to file
INFO:tensorflow:Added 253 to file
INFO:tensorflow:Added 254 to file
INFO:tensorflow:Added 255 to file
INFO:tensorflow:Added 256 to file
INFO:tensorflow:Added 257 to file
INFO:tensorflow:Added 258 to file
INFO:tensorflow:Added 259 to file
INFO:tensorflo

INFO:tensorflow:Added 472 to file
INFO:tensorflow:Added 473 to file
INFO:tensorflow:Added 474 to file
INFO:tensorflow:Added 475 to file
INFO:tensorflow:Added 476 to file
INFO:tensorflow:Added 477 to file
INFO:tensorflow:Added 478 to file
INFO:tensorflow:Added 479 to file
INFO:tensorflow:Added 480 to file
INFO:tensorflow:Added 481 to file
INFO:tensorflow:Added 482 to file
INFO:tensorflow:Added 483 to file
INFO:tensorflow:Added 484 to file
INFO:tensorflow:Added 485 to file
INFO:tensorflow:Added 486 to file
INFO:tensorflow:Added 487 to file
INFO:tensorflow:Added 488 to file
INFO:tensorflow:Added 489 to file
INFO:tensorflow:Added 490 to file
INFO:tensorflow:Added 491 to file
INFO:tensorflow:Added 492 to file
INFO:tensorflow:Added 493 to file
INFO:tensorflow:Added 494 to file
INFO:tensorflow:Added 495 to file
INFO:tensorflow:Added 496 to file
INFO:tensorflow:Added 497 to file
INFO:tensorflow:Added 498 to file
INFO:tensorflow:Added 499 to file
INFO:tensorflow:Added 500 to file
INFO:tensorflo

INFO:tensorflow:Added 713 to file
INFO:tensorflow:Added 714 to file
INFO:tensorflow:Added 715 to file
INFO:tensorflow:Added 716 to file
INFO:tensorflow:Added 717 to file
INFO:tensorflow:Added 718 to file
INFO:tensorflow:Added 719 to file
INFO:tensorflow:Added 720 to file
INFO:tensorflow:Added 721 to file
INFO:tensorflow:Added 722 to file
INFO:tensorflow:Added 723 to file
INFO:tensorflow:Added 724 to file
INFO:tensorflow:Added 725 to file
INFO:tensorflow:Added 726 to file
INFO:tensorflow:Added 727 to file
INFO:tensorflow:Added 728 to file
INFO:tensorflow:Added 729 to file
INFO:tensorflow:Added 730 to file
INFO:tensorflow:Added 731 to file
INFO:tensorflow:Added 732 to file
INFO:tensorflow:Added 733 to file
INFO:tensorflow:Added 734 to file
INFO:tensorflow:Added 735 to file
INFO:tensorflow:Added 736 to file
INFO:tensorflow:Added 737 to file
INFO:tensorflow:Added 738 to file
INFO:tensorflow:Added 739 to file
INFO:tensorflow:Added 740 to file
INFO:tensorflow:Added 741 to file
INFO:tensorflo

INFO:tensorflow:Added 954 to file
INFO:tensorflow:Added 955 to file
INFO:tensorflow:Added 956 to file
INFO:tensorflow:Added 957 to file
INFO:tensorflow:Added 958 to file
INFO:tensorflow:Added 959 to file
INFO:tensorflow:Added 960 to file
INFO:tensorflow:Added 961 to file
INFO:tensorflow:Added 962 to file
INFO:tensorflow:Added 963 to file
INFO:tensorflow:Added 964 to file
INFO:tensorflow:Added 965 to file
INFO:tensorflow:Added 966 to file
INFO:tensorflow:Added 967 to file
INFO:tensorflow:Added 968 to file
INFO:tensorflow:Added 969 to file
INFO:tensorflow:Added 970 to file
INFO:tensorflow:Added 971 to file
INFO:tensorflow:Added 972 to file
INFO:tensorflow:Added 973 to file
INFO:tensorflow:Added 974 to file
INFO:tensorflow:Added 975 to file
INFO:tensorflow:Added 976 to file
INFO:tensorflow:Added 977 to file
INFO:tensorflow:Added 978 to file
INFO:tensorflow:Added 979 to file
INFO:tensorflow:Added 980 to file
INFO:tensorflow:Added 981 to file
INFO:tensorflow:Added 982 to file
INFO:tensorflo

INFO:tensorflow:Added 1190 to file
INFO:tensorflow:Added 1191 to file
INFO:tensorflow:Added 1192 to file
INFO:tensorflow:Added 1193 to file
INFO:tensorflow:Added 1194 to file
INFO:tensorflow:Added 1195 to file
INFO:tensorflow:Added 1196 to file
INFO:tensorflow:Added 1197 to file
INFO:tensorflow:Added 1198 to file
INFO:tensorflow:Added 1199 to file
INFO:tensorflow:Added 1200 to file
INFO:tensorflow:Added 1201 to file
INFO:tensorflow:Added 1202 to file
INFO:tensorflow:Added 1203 to file
INFO:tensorflow:Added 1204 to file
INFO:tensorflow:Added 1205 to file
INFO:tensorflow:Added 1206 to file
INFO:tensorflow:Added 1207 to file
INFO:tensorflow:Added 1208 to file
INFO:tensorflow:Added 1209 to file
INFO:tensorflow:Added 1210 to file
INFO:tensorflow:Added 1211 to file
INFO:tensorflow:Added 1212 to file
INFO:tensorflow:Added 1213 to file
INFO:tensorflow:Added 1214 to file
INFO:tensorflow:Added 1215 to file
INFO:tensorflow:Added 1216 to file
INFO:tensorflow:Added 1217 to file
INFO:tensorflow:Adde

INFO:tensorflow:Added 1425 to file
INFO:tensorflow:Added 1426 to file
INFO:tensorflow:Added 1427 to file
INFO:tensorflow:Added 1428 to file
INFO:tensorflow:Added 1429 to file
INFO:tensorflow:Added 1430 to file
INFO:tensorflow:Added 1431 to file
INFO:tensorflow:Added 1432 to file
INFO:tensorflow:Added 1433 to file
INFO:tensorflow:Added 1434 to file
INFO:tensorflow:Added 1435 to file
INFO:tensorflow:Added 1436 to file
INFO:tensorflow:Added 1437 to file
INFO:tensorflow:Added 1438 to file
INFO:tensorflow:Added 1439 to file
INFO:tensorflow:Added 1440 to file
INFO:tensorflow:Added 1441 to file
INFO:tensorflow:Added 1442 to file
INFO:tensorflow:Added 1443 to file
INFO:tensorflow:Added 1444 to file
INFO:tensorflow:Added 1445 to file
INFO:tensorflow:Added 1446 to file
INFO:tensorflow:Added 1447 to file
INFO:tensorflow:Added 1448 to file
INFO:tensorflow:Added 1449 to file
INFO:tensorflow:Added 1450 to file
INFO:tensorflow:Added 1451 to file
INFO:tensorflow:Added 1452 to file
INFO:tensorflow:Adde

INFO:tensorflow:Added 1660 to file
INFO:tensorflow:Added 1661 to file
INFO:tensorflow:Added 1662 to file
INFO:tensorflow:Added 1663 to file
INFO:tensorflow:Added 1664 to file
INFO:tensorflow:Added 1665 to file
INFO:tensorflow:Added 1666 to file
INFO:tensorflow:Added 1667 to file
INFO:tensorflow:Added 1668 to file
INFO:tensorflow:Added 1669 to file
INFO:tensorflow:Added 1670 to file
INFO:tensorflow:Added 1671 to file
INFO:tensorflow:Added 1672 to file
INFO:tensorflow:Added 1673 to file
INFO:tensorflow:Added 1674 to file
INFO:tensorflow:Added 1675 to file
INFO:tensorflow:Added 1676 to file
INFO:tensorflow:Added 1677 to file
INFO:tensorflow:Added 1678 to file
INFO:tensorflow:Added 1679 to file
INFO:tensorflow:Added 1680 to file
INFO:tensorflow:Added 1681 to file
INFO:tensorflow:Added 1682 to file
INFO:tensorflow:Added 1683 to file
INFO:tensorflow:Added 1684 to file
INFO:tensorflow:Added 1685 to file
INFO:tensorflow:Added 1686 to file
INFO:tensorflow:Added 1687 to file
INFO:tensorflow:Adde

INFO:tensorflow:Added 1895 to file
INFO:tensorflow:Added 1896 to file
INFO:tensorflow:Added 1897 to file
INFO:tensorflow:Added 1898 to file
INFO:tensorflow:Added 1899 to file
INFO:tensorflow:Added 1900 to file
INFO:tensorflow:Added 1901 to file
INFO:tensorflow:Added 1902 to file
INFO:tensorflow:Added 1903 to file
INFO:tensorflow:Added 1904 to file
INFO:tensorflow:Added 1905 to file
INFO:tensorflow:Added 1906 to file
INFO:tensorflow:Added 1907 to file
INFO:tensorflow:Added 1908 to file
INFO:tensorflow:Added 1909 to file
INFO:tensorflow:Added 1910 to file
INFO:tensorflow:Added 1911 to file
INFO:tensorflow:Added 1912 to file
INFO:tensorflow:Added 1913 to file
INFO:tensorflow:Added 1914 to file
INFO:tensorflow:Added 1915 to file
INFO:tensorflow:Added 1916 to file
INFO:tensorflow:Added 1917 to file
INFO:tensorflow:Added 1918 to file
INFO:tensorflow:Added 1919 to file
INFO:tensorflow:Added 1920 to file
INFO:tensorflow:Added 1921 to file
INFO:tensorflow:Added 1922 to file
INFO:tensorflow:Adde

INFO:tensorflow:Added 2130 to file
INFO:tensorflow:Added 2131 to file
INFO:tensorflow:Added 2132 to file
INFO:tensorflow:Added 2133 to file
INFO:tensorflow:Added 2134 to file
INFO:tensorflow:Added 2135 to file
INFO:tensorflow:Added 2136 to file
INFO:tensorflow:Added 2137 to file
INFO:tensorflow:Added 2138 to file
INFO:tensorflow:Added 2139 to file
INFO:tensorflow:Added 2140 to file
INFO:tensorflow:Added 2141 to file
INFO:tensorflow:Added 2142 to file
INFO:tensorflow:Added 2143 to file
INFO:tensorflow:Added 2144 to file
INFO:tensorflow:Added 2145 to file
INFO:tensorflow:Added 2146 to file
INFO:tensorflow:Added 2147 to file
INFO:tensorflow:Added 2148 to file
INFO:tensorflow:Added 2149 to file
INFO:tensorflow:Added 2150 to file
INFO:tensorflow:Added 2151 to file
INFO:tensorflow:Added 2152 to file
INFO:tensorflow:Added 2153 to file
INFO:tensorflow:Added 2154 to file
INFO:tensorflow:Added 2155 to file
INFO:tensorflow:Added 2156 to file
INFO:tensorflow:Added 2157 to file
INFO:tensorflow:Adde

In [12]:
def read_contents(file):
    with open(file, 'r') as f:
        summary = f.read()
    return summary



def get_file_names(path,root):
    name = os.path.split(path)[1]
    example_id = name.split('_')[0]
    article_file = os.path.join(root, f"reference/{example_id}_article.txt")
    ref_file = os.path.join(root, f"reference/{example_id}_reference.txt")
    return example_id,article_file,ref_file


def get_analysis_data(input_df,model,vocab,log=False):
    
    final_data = []
    path = f'/home/ubuntu/W266/final_0/W266_Final/{model}/saved/decode'
    summary_dir = os.path.join(path, "generated/*")
    filelist = glob.glob(summary_dir) 
    column_list = ['model','Article OOV','Generated','Copied','2-Gram(repeat)','3-Gram(repeat)','4-Gram(repeat)']

    
    for f in filelist:
        data = {}
        example_id,article_f,reference_f = get_file_names(f,path)
        summary = read_contents(f)
        article = read_contents(article_f)
        reference = read_contents(reference_f)
        data['hyp'] = summary
        data['ref'] = reference
        final_data.append(data)
    return final_data        


def rouge_eval(data):

    hyps, refs = map(list, zip(*[[d['hyp'], d['ref']] for d in data]))
    rouge = Rouge()
    scores = rouge.get_scores(hyps, refs, avg=True)    

    print('======================================================================')
    print('======================================================================')
    print('ROUGE SCORES')
    print(scores)
    print('======================================================================')
    print('======================================================================')


In [13]:
rouge_eval(get_analysis_data(None,'model_3',None))

ROUGE SCORES
{'rouge-1': {'f': 0.16811019639545688, 'p': 0.29538138124491375, 'r': 0.12488278778749307}, 'rouge-2': {'f': 0.020109340540391516, 'p': 0.032864420504544606, 'r': 0.01572687152206284}, 'rouge-l': {'f': 0.12676682056060748, 'p': 0.27828727509597573, 'r': 0.11730069933854562}}
